In [125]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
import time

In [126]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

In [127]:
def _calculate(self, X, y, categorical, metafeatures, helpers):
        import sklearn.naive_bayes

        if len(y.shape) == 1 or y.shape[1] == 1:
            kf = StratifiedKFold(y, n_folds=10)
        else:
            kf = KFold(y.shape[0], n_folds=10)

        accuracy = 0.
        for train, test in kf:
            nb = sklearn.naive_bayes.GaussianNB()

            if len(y.shape) == 1 or y.shape[1] == 1:
                nb.fit(X[train], y[train])
            else:
                nb = OneVsRestClassifier(nb)
                nb.fit(X[train], y[train])

            predictions = nb.predict(X[test])
            accuracy += sklearn.metrics.accuracy_score(predictions, y[test])
        return accuracy / 10

In [148]:
class clust():
    @timeit
    def classify(self, model=LogisticRegression(random_state=42)):
        model.fit(self.X_train, self.y_train)
        y_pred = model.predict(self.X_test)
        print('Accuracy: {}'.format(accuracy_score(self.y_test, y_pred)))
    @timeit
    def Kmeans(self, output='add'):
        n_clusters = len(np.unique(self.y_train))
        clf = KMeans(n_clusters = n_clusters, random_state=42)
        clf.fit(self.X_train)
        y_labels_train = clf.labels_
        y_labels_test = clf.predict(self.X_test)
        if output == 'add':
            self.X_train['km_clust'] = y_labels_train
            self.X_test['km_clust'] = y_labels_test
        elif output == 'replace':
            self.X_train = y_labels_train[:, np.newaxis]
            self.X_test = y_labels_test[:, np.newaxis]
        else:
            raise ValueError('output should be either add or replace')
        self.classify()
        return self
    def _load_data(self, sklearn_load_ds):
        import sklearn.naive_bayes
        data = sklearn_load_ds
        X = pd.DataFrame(data.data)
        y = data.target
        
        kf = KFold(n_splits=10)
        accuracy = 0
        for train_index, test_index in kf.split(X):
            self.X_train, self.X_test = X.iloc[train_index], X.iloc[test_index]
            self.y_train, self.y_test = y.iloc[train_index], y.iloc[test_index]
            self.Kmeans(output='add')
            return self
            #print("TRAIN:", train_index, "TEST:", test_index)
            #self.X_train, self.X_test = X.ix[train_index], X.ix[test_index]
            #self.y_train, self.y_test = y.ix[train_index], y.ix[test_index]
            #nb = sklearn.naive_bayes.GaussianNB()
            #nb = OneVsRestClassifier(nb)
            #nb.fit(self.X_train, self.y_train)
            #predictions = nb.predict(self.X_test)
            #accuracy += sklearn.metrics.accuracy_score(predictions, self.y_test)
            #print('Accuracy: {}'.format(accuracy / 10))
        
    def __init__(self, sklearn_load_ds):
        self._load_data(sklearn_load_ds)
    
    

    

In [149]:
class loaddata:
    def __init__(self, data, target):
        self.data = data
        self.target = target

In [150]:
#reading Data
X_data = pd.read_csv('../data/raw/Data Cleaning.csv')
y_data = pd.read_excel('../data/raw/Training outputs.xlsx')

X_data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
y_data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [152]:
load_data = loaddata(X_data,y_data)
clust(load_data)

C:\ProgramData\Anaconda3\envs\opencv\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\envs\opencv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\envs\opencv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this war

Accuracy: 0.7207792207792207
'classify'  10738.61 ms
'Kmeans'  11539.66 ms


C:\ProgramData\Anaconda3\envs\opencv\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
